In [2]:
import torch
import transformers
import pandas as pd
import numpy as np
import torch
import os
import sys
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, precision_score, recall_score, precision_recall_curve, auc

In [3]:
from transformers import AutoModel
from transformers import AutoTokenizer
#seyonec/PubChem10M_SMILES_BPE_450k
from transformers import RobertaModel,BertModel

In [4]:
model = RobertaModel.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

train = pd.read_csv('../input/paper-classcification/train.csv', sep='\t')
test = pd.read_csv('../input/paper-classcification/test.csv', sep='\t')
sub = pd.read_csv('../input/paper-classcification/sample_submit.csv')

# 拼接title与abstract
train['text'] = train['title'] + ' ' + train['abstract']
test['text'] = test['title'] + ' ' + test['abstract']

label_id2cate = dict(enumerate(train.categories.unique()))
label_cate2id = {value: key for key, value in label_id2cate.items()}

train['label'] = train['categories'].map(label_cate2id)

train = train[['text', 'label']]
train_y = train["label"]
# train_df = train[['text', 'label']][:45000]
# eval_df = train[['text', 'label']][45000:]


In [7]:
index = np.random.permutation(range(len(train)))
train_data = train.iloc[index[:int(0.9*len(train))]].reset_index(drop = True)
val_data = train.iloc[index[int(0.9*len(train)):]].reset_index(drop = True)

In [8]:
from torch.utils.data import Dataset,DataLoader,TensorDataset
class My_dataset(Dataset):
    def __init__(self,data_len):
        super(My_dataset,self).__init__()
            
        self.data = data_len
    def __getitem__(self,idx):
#         label = self.data.iloc[idx][2]
#         mol = self.data.iloc[idx][0]
#         pro = self.data.iloc[idx][1]
#         pro = ' '.join(pro)
#         mol_inputs = tokenizer_mol(mol)['input_ids']
#         mol_inputs_mask = tokenizer_mol(mol)['attention_mask']
#         pro_inputs = tokenizer_pro(pro)['input_ids']
#         pro_inputs_mask = tokenizer_pro(pro)['attention_mask']
#         pro_inputs_type = tokenizer_pro(pro)['token_type_ids']
        
        return self.data[idx]#mol,'_'.join(pro),label#torch.tensor(),torch.tensor(),torch.tensor(tss),torch.tensor(feats,dtype = torch.float32)
    def __len__(self):
        return len(self.data)
#class my_dataset(nn.)

In [51]:
val_dataloader = DataLoader(list(range(len(val_data))),batch_size = 128,num_workers = 0,shuffle = True)
train_dataloader = DataLoader(list(range(len(train_data))),batch_size = 128,num_workers = 0,shuffle = True)
test_dataloader = DataLoader(list(range(len(test))),batch_size = 128,num_workers = 0,shuffle = False)

In [52]:
class MLP(nn.Module):
    def __init__(self,dim_in,dim_hidden,dim_out):
        super(MLP,self).__init__()
        self.fc_1 = nn.Linear(dim_in,dim_hidden)
        self.fc_2 = nn.Linear(dim_hidden,dim_out)
    def forward(self,data):
        return self.fc_2(torch.relu(self.fc_1(data)))#return logits for crosss_entry

In [ ]:
modelmlp = MLP(768,768*2,39)
Loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modelmlp.parameters(),lr = 0.001)
epochs = 100
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
best_acc = 0
patience = 5
counter = 0
for epoch in range(1,epochs+1):
    print('strating training!!')
    train_loss = []
    val_loss = []
    #test_loss = 0
    S = []
    T = []
    S_train = []
    T_train = []
    modelmlp.to(device)
    modelmlp.train()
    model.to(device)
    model.eval()
    pbar = tqdm(train_dataloader)
    for batch in pbar:
        batch_data = train_data.iloc[batch].copy()
        #batch_data[1] = batch_data[1].apply(lambda x : re.sub(r"[UZOB]", "X", x))
        # mol_batch = list(batch_data[0].values)
        # pro_batch = list(batch_data[1].values)
        text_batch = list(batch_data['text'].values)
        label_batch = torch.tensor(list(batch_data['label'].values))
#         model_mol.eval()
#         model_pro.eval()
        with torch.no_grad():
            #size = [bs,768]
            tokenizer_input = tokenizer(text_batch,return_tensors = 'pt',padding = True,max_length = 512,truncation = True)
            input_ids,att_mask = tokenizer_input['input_ids'],tokenizer_input['attention_mask']
            model_out = model(input_ids.to(device),att_mask.to(device))['pooler_output']
            # #size = [bs,1024]
            # pro_out = model_pro(**tokenizer_pro(pro_batch,return_tensors = 'pt',padding = True,max_length = 1500,truncation = True))['pooler_output']
        # model_input = torch.cat([mol_out,pro_out],dim = 1)#bs,1792
        model_out = modelmlp(model_out.to(device))
        loss = Loss(model_out,label_batch.to(device))
        scores = torch.softmax(model_out,dim = 1).cpu().detach().numpy()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
        pbar.update()
        pbar.set_description('train_loss: {}'.format(np.sum(train_loss)/len(train_loss)))
        S_train.extend(scores)
        T_train.extend(label_batch.cpu().detach().numpy())
    AUC = roc_auc_score(T_train, S_train,multi_class='ovo')
#     tpr, fpr, _ = precision_recall_curve(T_train, S_train)
#     PRC = auc(fpr, tpr)
    #print(AUC, PRC)
    train_acc = np.sum(np.argmax(S_train,axis = 1) == T_train)/len(T_train)
    print('train_loss:, ',np.sum(train_loss)/len(train_loss),'AUC: ',AUC,'train_acc: ',train_acc) 
    
#val state
    modelmlp.eval()
    pbar = tqdm(val_dataloader)
    for batch in pbar:

        batch_data = val_data.iloc[batch].copy()
        #batch_data[1] = batch_data[1].apply(lambda x : re.sub(r"[UZOB]", "X", x))
        # mol_batch = list(batch_data[0].values)
        # pro_batch = list(batch_data[1].values)
        text_batch = list(batch_data['text'].values)
        label_batch = torch.tensor(list(batch_data['label'].values))
        with torch.no_grad():
            #size = [bs,768]
            # mol_out = model_mol(**tokenizer_mol(mol_batch,return_tensors = 'pt',padding = True,max_length = 512,truncation = True))['pooler_output']
            #size = [bs,1024]
            tokenizer_input = tokenizer(text_batch,return_tensors = 'pt',padding = True,max_length = 512,truncation = True)
            input_ids,att_mask = tokenizer_input['input_ids'],tokenizer_input['attention_mask']
            model_out = model(input_ids.to(device),att_mask.to(device))['pooler_output']
            # model_input = torch.cat([mol_out,pro_out],dim = 1)#bs,1792
            model_out = modelmlp(model_out.to(device))
            loss = Loss(model_out,label_batch.to(device))
            #scores = torch.softmax(model_out,dim = 1)[:,1]
            val_loss.append(loss.item())
            scores = torch.softmax(model_out,dim = 1).cpu().detach().numpy()
            S.extend(scores)
            T.extend(label_batch.cpu().detach().numpy())
            pbar.set_description('val_loss: {}'.format(np.sum(val_loss)/len(val_loss)))
            pbar.update()
    AUC = roc_auc_score(T, S,multi_class='ovo')
#     tpr, fpr, _ = precision_recall_curve(T, S)
#     PRC = auc(fpr, tpr)
    val_acc = np.sum(np.argmax(S,axis = 1) == T)/len(T)
    counter +=1
    print('val_loss: ',np.sum(val_loss)/len(val_loss),'AUC: ',AUC,'val_acc: ',val_acc) 
    if val_acc > best_acc:
        counter = 0
        best_acc = val_acc
        torch.save({'model':modelmlp.state_dict(),
                   'epoch':epoch,
                   'optimizer':optimizer.state_dict(),
                   'best_acc':best_acc,
                   'val_loss':val_loss},'./my_pretrain_model.pth')
    if counter > patience:
        print('early stop in %d'%epoch)
        break
#test 
modelmlp.eval()
pbar = tqdm(test_dataloader)
predictions = []
for batch in pbar:

    batch_data = test.iloc[batch].copy()
    #batch_data[1] = batch_data[1].apply(lambda x : re.sub(r"[UZOB]", "X", x))
    # mol_batch = list(batch_data[0].values)
    # pro_batch = list(batch_data[1].values)
    text_batch = list(batch_data['text'].values)
    #label_batch = torch.tensor(list(batch_data['label'].values))
    with torch.no_grad():
        #size = [bs,768]
        # mol_out = model_mol(**tokenizer_mol(mol_batch,return_tensors = 'pt',padding = True,max_length = 512,truncation = True))['pooler_output']
        #size = [bs,1024]
        tokenizer_input = tokenizer(text_batch,return_tensors = 'pt',padding = True,max_length = 512,truncation = True)
        input_ids,att_mask = tokenizer_input['input_ids'],tokenizer_input['attention_mask']
        model_out = model(input_ids.to(device),att_mask.to(device))['pooler_output']
        # model_input = torch.cat([mol_out,pro_out],dim = 1)#bs,1792
        model_out = modelmlp(model_out.to(device))
#             loss = Loss(model_out,label_batch.to(device))
        #scores = torch.softmax(model_out,dim = 1)[:,1]
#         val_loss.append(loss.item())
        scores = torch.softmax(model_out,dim = 1).cpu().detach().numpy()
        prediction = np.argmax(scores,axis = 1)
        predictions.extend(prediction)
        pbar.update()
sub['categories'] = predictions
sub['categories'] = sub['categories'].map(label_id2cate)
sub.to_csv('./submit.csv', index=False)
print('done')


  0%|          | 0/352 [00:00<?, ?it/s]

strating training!!


  0%|          | 0/40 [00:00<?, ?it/s]

train_loss:,  2.8612953647971153 AUC:  0.5999713844268169 train_acc:  0.24322222222222223


  0%|          | 0/352 [00:00<?, ?it/s]

...

  0%|          | 0/352 [00:00<?, ?it/s]

val_loss:  1.3970359116792679 AUC:  0.9306838563683459 val_acc:  0.5892
strating training!!


  0%|          | 0/40 [00:00<?, ?it/s]

train_loss:,  1.3995899178765037 AUC:  0.9224821854835608 train_acc:  0.5950666666666666


val_loss: 1.309428258375688:  38%|███▊      | 15/40 [00:23<00:47,  1.89s/it] 

In [34]:
torch.load('./my_pretrain_model.pth')

{'model': OrderedDict([('fc_1.weight',
               tensor([[-0.0203, -0.0073,  0.0152,  ...,  0.0065, -0.0029,  0.0275],
                       [-0.0146,  0.0135,  0.0076,  ..., -0.0159, -0.0069, -0.0150],
                       [ 0.0072, -0.0293, -0.0082,  ...,  0.0340,  0.0354,  0.0096],
                       ...,
                       [-0.0028,  0.0049, -0.0280,  ..., -0.0045, -0.0193,  0.0233],
                       [ 0.0094,  0.0129,  0.0210,  ..., -0.0098, -0.0001,  0.0292],
                       [ 0.0035, -0.0009, -0.0239,  ...,  0.0039,  0.0044,  0.0027]],
                      device='cuda:0')),
              ('fc_1.bias',
               tensor([ 0.0277, -0.0112,  0.0122,  ..., -0.0228,  0.0244, -0.0321],
                      device='cuda:0')),
              ('fc_2.weight',
               tensor([[ 0.0175,  0.0259, -0.0117,  ...,  0.0216, -0.0104,  0.0215],
                       [-0.0250, -0.0195, -0.0165,  ...,  0.0144,  0.0197,  0.0213],
                       [ 0.0

In [40]:
pd.read_csv('./submit.csv')['categories'].unique()

array(['cs.CV'], dtype=object)

In [ ]:
y_true = [1,2,3,4,5,6,7,8]

In [50]:
train.groupby('label').size()/len(train)

label
0     0.08520
1     0.01408
2     0.01074
3     0.22076
4     0.02704
5     0.05018
6     0.01540
7     0.03768
8     0.01046
9     0.05596
10    0.00726
11    0.06436
12    0.05412
13    0.03880
14    0.01366
15    0.03482
16    0.02584
17    0.00628
18    0.01682
19    0.01206
20    0.01354
21    0.01886
22    0.00420
23    0.01968
24    0.00460
25    0.00938
26    0.01438
27    0.01996
28    0.03988
29    0.02456
30    0.00724
31    0.00522
32    0.00352
33    0.00278
34    0.00198
35    0.00280
36    0.00344
37    0.00210
38    0.00036
dtype: float64

In [12]:
sub

,paperid,categories
0,test_00000,cs.CV
1,test_00001,cs.CV
2,test_00002,cs.CV
3,test_00003,cs.CV
4,test_00004,cs.CV
...,...,...
9995,test_09995,cs.CV
9996,test_09996,cs.CV
9997,test_09997,cs.CV
9998,test_09998,cs.CV
